In [1]:
import pandas as pd
from pprint import pprint
import ast

## 데이터 불러오기

In [ ]:
match_df = pd.read_csv('match_data_version1.csv')
match_df.shape

### 형태 보기

In [ ]:
match_df.shape[0]

In [ ]:
list(match_df.columns)

In [ ]:
match_df.head()

In [ ]:
match_df = match_df.drop(['Unnamed: 0','gameCreation','gameMode','gameType','mapId','platformId','seasonId','status.message','status.status_code'], axis=1)

In [ ]:
match_df.head()

In [ ]:
pprint(eval(match_df['participantIdentities'][1]))

In [ ]:
pprint(eval(match_df['participants'][1]))

In [ ]:
a = ast.literal_eval(match_df['participants'][1])
type(a[0])

## 함수 만들기 

In [ ]:
column_list=list(match_df.columns)
value_list = []

def solve_dict_column(dict_to_iterate, upper_key=None, pID = None):            # 하드 코딩, 추후 수정 필요
    global column_list
    global value_list
    for key, value in dict_to_iterate.items():
        if key == 'participantId':
            pID = 'P' + str(value)
        
        if type(value) == dict:
            solve_dict_column(value, key, pID)
        else:
            if upper_key == None:
                column_list.append(pID+ '_' + key)

            else :
                column_list.append(pID + '_' + str(upper_key) + '_' + key)
            value_list.append(value)

In [ ]:
final = pd.DataFrame()

n = match_df.shape[0]
except_list = []
for i in range(n):
    temp = pd.DataFrame()
    column_list = []
    value_list = []
    if (len(eval(match_df.iloc[i]['participants'])) ==10) and  (len(eval(match_df.iloc[i]['participantIdentities'])) == 10):
        for x,y in enumerate([0,1,2,5]):
            column_list.append(match_df.columns[y])
            value_list.append(match_df.loc[i][y])
        for j in range(10):
            solve_dict_column(ast.literal_eval(match_df['participants'][i])[j])
            solve_dict_column(ast.literal_eval(match_df['participantIdentities'][i])[j])

        d=[]
        new_d = [] # 중복된 원소만 넣는 리스트
        for k in column_list:
            if k not in d: # 처음 등장한 원소
                d.append(k)
            else:
                if k not in new_d: # 이미 중복 원소로 판정된 경우는 제외
                    new_d.append(k)

        new_d # [1, 2] # 2회 이상 등장한 값들만 담긴 리스트
        
        temp = pd.DataFrame(data=[value_list], columns=column_list)
        temp = temp.drop(new_d, axis=1)

        final = pd.concat([final, temp], axis=0)
    else:
        except_list.append(i)

    if (i%1000) == 0:
        print(i)


final.head()

In [ ]:
final.to_csv("make_final.csv", mode='w')